<a href="https://colab.research.google.com/github/smaali1rim/GenAI-with-Google-learn-guide/blob/main/Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 1 - Prompting

### Install the SDK¶

In [1]:
!pip install -U -q "google-generativeai>=0.8.3"

In [2]:
import google.generativeai as genai
from IPython import get_ipython
from IPython.display import HTML, Markdown, display

### Set up the API key

In [20]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

### Choose a model

In [4]:
for model in genai.list_models():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/learnlm-1.5-pro-experimental
models/embedding-001
models/text-embedding-004
models/aqa


In [5]:
for model in genai.list_models():
  if model.name == 'models/gemini-1.5-flash':
    print(model)
    break

Model(name='models/gemini-1.5-flash',
      base_model_id='',
      version='001',
      display_name='Gemini 1.5 Flash',
      description=('Alias that points to the most recent stable version of Gemini 1.5 Flash, our '
                   'fast and versatile multimodal model for scaling across diverse tasks.'),
      input_token_limit=1000000,
      output_token_limit=8192,
      supported_generation_methods=['generateContent', 'countTokens'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=40)


# Explore generation parameters

### Output length

In [6]:
short_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(max_output_tokens=200))

response = short_model.generate_content('Write a 1000 word essay on the importance of olives in modern society.')
print(response.text)

## The Enduring Significance of Olives in Modern Society

The olive, a seemingly unassuming fruit, holds a position of profound importance in modern society, extending far beyond its culinary applications. Its significance is woven into the fabric of history, culture, economics, and even environmental sustainability. From the Mediterranean basin, where it originated, to global markets, the olive and its products – olive oil, olives for table consumption, and olive pomace – have shaped civilizations and continue to play a vital role in shaping our present and future.

The historical significance of the olive tree is undeniable.  Its cultivation dates back millennia, with evidence suggesting its domestication in the eastern Mediterranean around 6000 BC.  Ancient civilizations, including the Greeks and Romans, revered the olive tree as a symbol of peace, prosperity, and wisdom.  The olive branch, a universal emblem of peace, originates from the biblical story of Noah's Ark, solidifying it

### Temperature

In [7]:
from google.api_core import retry

high_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=2.0))


# When running lots of queries, it's a good practice to use a retry policy so your code
# automatically retries when hitting Resource Exhausted (quota limit) errors.
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=300)
}

for _ in range(5):
  response = high_temp_model.generate_content('Pick a random country... (respond in a single word)',
                                              request_options=retry_policy)
  if response.parts:
    print(response.text, '-' * 25)

Bhutan
 -------------------------
Kyrgyzstan
 -------------------------
Bhutan
 -------------------------
Bhutan
 -------------------------
Bhutan
 -------------------------


In [8]:
low_temp_model = genai.GenerativeModel(
    'gemini-1.5-flash',
    generation_config=genai.GenerationConfig(temperature=0.0))

for _ in range(5):
  response = low_temp_model.generate_content('Pick a random country... (respond in a single word)',
                                             request_options=retry_policy)
  if response.parts:
    print(response.text, '-' * 25)

Bhutan
 -------------------------
Bhutan
 -------------------------
Bhutan
 -------------------------
Bhutan
 -------------------------
Bhutan
 -------------------------


### Top-K and top-P

- Top-K sampling selects the top K most likely tokens from the model’s predicted
distribution. The higher top-K, the more creative and varied the model’s output; the
lower top-K, the more restive and factual the model’s output. A top-K of 1 is equivalent to
greedy decoding.

- Top-P sampling selects the top tokens whose cumulative probability does not exceed
a certain value (P). Values for P range from 0 (greedy decoding) to 1 (all tokens in the
LLM’s vocabulary).


In [9]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        # These are the default values for gemini-1.5-flash-001.
        temperature=1.0,
        top_k=64,
        top_p=0.95,
    ))

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = model.generate_content(story_prompt, request_options=retry_policy)
print(response.text)

Barnaby was a tabby cat of discerning tastes. He preferred the comfort of his plush cushion to the thrill of chasing mice, and the aroma of fresh tuna to the smell of dew-kissed grass. But Barnaby, unlike most cats, had a secret longing for adventure. It bubbled beneath his purrs and flickered in his emerald eyes, a yearning for a world beyond the familiar confines of his home.

One day, the window was left ajar, a lapse in the vigilance of Barnaby's human. The enticing scent of freedom wafted in, carrying with it the whisper of the unknown. Barnaby, unable to resist the call, slipped out into the world, his heart pounding with a mixture of excitement and trepidation.

The world was a sensory explosion – sights, sounds, smells, all unfamiliar and exhilarating. Barnaby, a seasoned indoor cat, found himself on a journey of discovery. He dodged playful squirrels in the park, marveled at the kaleidoscope of colors in a flower bed, and even had a close encounter with a grumpy old tomcat who

In [10]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        # These are the default values for gemini-1.5-flash-001.
        temperature=0.5,
        top_k=10,
        top_p=0.8,
    ))

story_prompt = "You are a creative writer. Write a short story about a cat who goes on an adventure."
response = model.generate_content(story_prompt, request_options=retry_policy)
print(response.text)

Bartholomew, a ginger tabby with a penchant for mischief and a disdain for routine, was bored. The sunbeams that usually danced across the living room floor were dull, the bird feeder outside was empty, and his human, Emily, was engrossed in a book, oblivious to his plight. This was unacceptable. Bartholomew, a cat of discerning taste, demanded excitement.

He surveyed the room, his green eyes landing on the open window. A gust of wind rustled the curtains, whispering tales of adventure. Bartholomew knew what he had to do. He leapt onto the windowsill, his claws finding purchase on the rough wood. With a graceful leap, he was outside.

The world was a symphony of smells and sounds. The scent of damp earth, the chirping of unseen birds, the distant rumble of traffic – it was intoxicating. Bartholomew, a seasoned hunter, stalked through the garden, his tail held high. He chased butterflies, stalked a plump robin (who thankfully took flight), and even engaged in a playful battle with a st

# Prompting

### Zero-shot

Zero-shot prompts are prompts that describe the request for the model directly.

In [11]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=5,
    ))

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

Sentiment: **POSITIVE**


#### Enum mode

The models are trained to generate text, and can sometimes produce more text than *we* may wish for. In the preceding example, the model will output the label, sometimes it can include a preceding "Sentiment" label, and without an output token limit, it may also add explanatory text afterwards.

The Gemini API has an [Enum mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Enum.ipynb) feature that allows us to constrain the output to a fixed set of values.

In [12]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"


model = genai.GenerativeModel(
    'gemini-1.5-flash-001',
    generation_config=genai.GenerationConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ))

response = model.generate_content(zero_shot_prompt, request_options=retry_policy)
print(response.text)

positive


### One-shot and few-shot

Providing an example of the expected response is a "one-shot" prompt. When  multiple examples are provided, it is a "few-shot" prompt.

In [13]:
model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ))

few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""

customer_order = "Give me a large with cheese & pineapple"


response = model.generate_content([few_shot_prompt, customer_order], request_options=retry_policy)
print(response.text)

```json
{
  "size": "large",
  "type": "normal",
  "ingredients": ["cheese", "pineapple"]
}
```



#### JSON mode

To provide control over the schema, and to ensure that we only receive JSON (with no other text or markdown), we can use the Gemini API's [JSON mode](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb). This forces the model to constrain decoding, such that token selection is guided by the supplied schema.

In [19]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str


model = genai.GenerativeModel(
    'gemini-1.5-flash-latest',
    generation_config=genai.GenerationConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ))

response = model.generate_content("Can I have a large dessert pizza with apple and chocolate")
print(response.text)

{"ingredients": ["apple", "chocolate"], "size": "large", "type": "dessert pizza"}


### Chain of Thought (CoT)

Chain-of-Thought prompting is a technique where we instruct the model to output intermediate reasoning steps, and it typically gets better results, especially when combined with few-shot examples.

In [15]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer directly."""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
response = model.generate_content(prompt, request_options=retry_policy)

print(response.text)

41



In [16]:
prompt = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""

response = model.generate_content(prompt, request_options=retry_policy)
print(response.text)

Here's how to solve this step-by-step:

1. **Partner's age when you were 4:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.

2. **Age difference:** The age difference between you and your partner is 12 - 4 = 8 years.

3. **Partner's current age:**  Since you are now 20, and the age difference remains constant, your partner is currently 20 + 8 = 28 years old.



### ReAct: Reason and act


In [17]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

In [18]:
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

model = genai.GenerativeModel('gemini-1.5-flash-latest')
react_chat = model.start_chat()

# You will perform the Action, so generate up to, but not including, the Observation.
config = genai.GenerationConfig(stop_sequences=["\nObservation"])

resp = react_chat.send_message(
    [model_instructions, example1, example2, question],
    generation_config=config,
    request_options=retry_policy)
print(resp.text)

Thought 1
I need to find the Transformers NLP paper and then find the authors and their ages to determine the youngest one.  I can't directly search for "youngest author" as the ages aren't readily available.

Action 1
<search>Transformers (deep learning model)</search>

